In [ ]:
#@title Clone Scaffold-Lab github repo 
import json, time, os, sys, glob

if not os.path.isdir("Scaffold-Lab"):
  os.system("git clone -q https://github.com/Immortals-33/Scaffold-Lab.git")
sys.path.append('/content/Scaffold-Lab')

In [8]:
# Check installed versions of pydantic and triton
import importlib

def check_version(package_name):
    try:
        pkg = importlib.import_module(package_name)
        version = getattr(pkg, '__version__', 'unknown')
        print(f"{package_name} version: {version}")
    except ImportError:
        print(f"{package_name} is not installed.")

check_version('pydantic')
check_version('triton')

pydantic version: 2.12.3
triton version: 3.5.0


In [ ]:
#@title Install Related Libraries
# !pip install pydantic==1.10.12
# !pip install triton==1.0.0
!pip install rootutils GPUtil biotite

!pip install --upgrade MDAnalysis
!pip install mdtraj tmtools
!pip install 
!pip install pandas dm-tree tabulate psutil
!pip install hydra-core --upgrade

!pip install hydra-joblib-launcher --upgrade
!pip install hydra-core --upgrade



#@title Install dependencies
#@markdown esmfold and openfold environment from ColabFold scripts
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'


# # Install esmfold
print("installing libs...")
os.system("pip install -q omegaconf pytorch_lightning biopython ml_collections einops py3Dmol modelcif")
os.system("pip install -q git+https://github.com/NVIDIA/dllogger.git")

print("installing openfold...")
# # install openfold
os.system(f"pip install -q git+https://github.com/sokrypton/openfold.git")
print("installing esmfold...")
# # install esmfold
os.system(f"pip install -q git+https://github.com/sokrypton/esm.git")


import time
print("downloading parameters...")
os.system("apt-get install aria2 -qq")

command = """
aria2c -q -x 16 https://colabfold.steineggerlab.workers.dev/esm/esmfold.model &
wait
"""
os.system(command)
print("\nAll parameters have been downloaded successfully!")

  Using cached rootutils-1.0.7-py3-none-any.whl.metadata (4.7 kB)
  Using cached GPUtil-1.4.0.tar.gz (5.5 kB)
  Preparing metadata (setup.py) ... done
Using cached rootutils-1.0.7-py3-none-any.whl (6.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 MB 67.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 107.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.1/35.1 MB 60.2 MB/s eta 0:00:00:00:01
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7393 sha256=1e2c3c0abef049919535eaaefc82804016dec5cdce18be2327b053097a4739a5
  Stored in directory: /root/.cache/pip/wheels/2b/4d/8f/55fb4f7b9b591891e8d3f72977c4ec6c7763b39c19f0861595
Successfully built GPUtil
  Using cached griddataformats-1.1.0-py3-none-any.whl.metadata (5.0 kB)
  Using cached mmtf_python-1.1.3-py2.py3-none-any.whl.metadata (1.2 kB)
  Using cached mda_xdrlib-0.2.0-py3-no

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 128.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 77.1 MB/s eta 0:00:00
  Using cached hydra_core-1.3.2-py3-none-any.whl.metadata (5.5 kB)
  Preparing metadata (setup.py) ... done
Using cached hydra_core-1.3.2-py3-none-any.whl (154 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.6/806.6 kB 33.6 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144555 sha256=b621aabf992fe07bb69273ceffe9809d0d513fb26b8f96ec4f86034e3e83def3
  Stored in directory: /root/.cache/pip/wheels/1a/97/32/461f837398029ad76911109f07047fde1d7b661a147c7c56d1
Successfully built antlr4-python3-runtime


  Using cached hydra_joblib_launcher-1.2.0-py3-none-any.whl.metadata (1.0 kB)
Using cached hydra_joblib_launcher-1.2.0-py3-none-any.whl (5.2 kB)
installing libs...
installing openfold...
installing esmfold...
downloading parameters...

All parameters have been downloaded successfully!


In [2]:
#@title Install PyMol
!pip install pymol-open-source


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 11.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 166.9 MB/s eta 0:00:00


In [4]:
#@title Config

mode = "motif_scaffolding" #@param ["unconditional", "motif_scaffolding"]
#@markdown - task name : unconditional generation or motif scaffolding
query_pdb_folder = '/content/Scaffold-Lab/demo/motif_scaffolding/2KL8/' #@param {type:"string"}
#@markdown - customised pdb file path, you can direct to your colab path
contig_str = 'none' #@param {type:"string"}
#@markdown - **only motif scaffolding** : information on motifs and scaffolds of each designed protein such as "A1-7/20-20/A28-79"
contig_csv = '/content/Scaffold-Lab/demo/motif_scaffolding/motif_info.csv' #@param {type:"string"}
#@markdown - **only motif scaffolding** : information on motifs and scaffolds of each designed protein, only used in motif_scaffolding problem.This is particularly important when sample length-variable scaffolds for a single case, for different samples correspond to different motif regions.
native_pdb_folder = '/content/Scaffold-Lab/demo/motif_scaffolding/reference_pdbs' #@param {type:"string"}
#@markdown - **only motif scaffolding** : ground truth pdb folder 
metric = "scRMSD_pAE_motifRMSD" #@param ["scRMSD_pLDDT_motifRMSD", "scRMSD_pAE_motifRMSD"]
#@markdown - the in silico computation metric for successful design

In [21]:
#@title Fixed backbone design and Refolding
import os
# 清除 MPLBACKEND 环境变量，避免 matplotlib 后端兼容性问题
if 'MPLBACKEND' in os.environ:
  del os.environ['MPLBACKEND']

# 切换到项目目录，让 rootutils 能找到 .git 文件夹
%cd /content/Scaffold-Lab

if mode == "motif_scaffolding":
  !python /content/Scaffold-Lab/scaffold_lab/motif_scaffolding/motif_refolding.py inference.backbone_pdb_dir={query_pdb_folder} inference.output_dir='./' inference.motif_csv_path={contig_csv}
elif mode == "unconditional":
  !python /content/Scaffold-Lab/scaffold_lab/unconditional/refolding.py inference.backbone_pdb_dir={query_pdb_folder} inference.output_dir='./'

/content/Scaffold-Lab
Error executing job with overrides: ['inference.backbone_pdb_dir=/content/Scaffold-Lab/demo/motif_scaffolding/2KL8/', 'inference.output_dir=./', 'inference.motif_csv_path=/content/Scaffold-Lab/demo/motif_scaffolding/motif_info.csv']
Traceback (most recent call last):
  File "/content/Scaffold-Lab/scaffold_lab/motif_scaffolding/motif_refolding.py", line 1052, in run
    raise ValueError("The 'foldseek_database' must be specified in the configuration.")
ValueError: The 'foldseek_database' must be specified in the configuration.

Set the environment variable HYDRA_FULL_ERROR=1 for a complete stack trace.
